In [6]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, welch, find_peaks
from scipy.stats import kurtosis, skew, scoreatpercentile, entropy

import pywt

!pip install python_speech_features
import python_speech_features as psf

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=330b65c344e3efc604da0a83546aa3fc222666e84e6ada274984079d6dac6b9c
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features
Mounted at /content/drive


In [7]:
# load_data
sampling_rate = 100  # Hz
duration = 10

data = np.load('/content/drive/MyDrive/Data/simu_0.1_90_180R.npy')
data_clean = np.load('/content/drive/MyDrive/Data/simu_0_90_180R.npy')
data_clean_NoRR = np.load('/content/drive/MyDrive/Data/simu_0_90_180_NoRR.npy')


labels, signals =  data[:, -6:], data[:, :1000]
signals_clean =  data_clean[:, :1000]
signals_clean_NoRR =  data_clean_NoRR[:, :1000]

In [8]:
def statistics_features(signal):
    features = {}

    features['Maximum'] = np.max(signal)
    features['Minimum'] = np.min(signal)
    features['Mean'] = np.mean(signal)
    features['Median'] = np.median(signal)
    features['Variance'] = np.var(signal)
    features['Kurtosis'] = kurtosis(signal)
    features['Skewness'] = skew(signal)
    features['0.75Percentile'] = scoreatpercentile(signal, 75)
    features['0.25Percentile'] = scoreatpercentile(signal, 25)

    return features


In [9]:
from scipy.stats import entropy

def calculate_rms(signal):
    rms = np.sqrt(np.mean(np.square(signal)))
    return rms

def calculate_zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zcr_rate = len(zero_crossings) / (len(signal) - 1)
    return zcr_rate


def calculate_entropy(signal):
    prob_distribution = np.histogram(signal, bins=10, density=True)[0]
    ent = entropy(prob_distribution)
    return ent


def calculate_centroid(signal):
    centroid = np.sum(np.arange(len(signal)) * np.abs(signal)) / np.sum(np.abs(signal))
    return centroid

def calculate_mean_absolute_differences(signal):
    mad = np.mean(np.abs(np.diff(signal)))
    return mad

def calculate_mean_differences(signal):
    mean_diff = np.mean(np.diff(signal))
    return mean_diff

def calculate_median_absolute_differences(signal):
    med_abs_diff = np.median(np.abs(np.diff(signal)))
    return med_abs_diff

def calculate_median_differences(signal):
    med_diff = np.median(np.diff(signal))
    return med_diff

def calculate_sum_of_absolute_differences(signal):
    sad = np.sum(np.abs(np.diff(signal)))
    return sad

def calculate_absolute_energy(signal):
    abs_energy = np.sum(np.square(signal))
    return abs_energy

def calculate_total_energy(signal):
    total_energy = np.sum(np.square(signal)) / len(signal)
    return total_energy


In [10]:



def calculate_spectral_entropy(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    prob_distribution = Pxx / np.sum(Pxx)
    spec_entropy = entropy(prob_distribution)
    return spec_entropy


def calculate_spectral_centroid(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    spectral_centroid = np.sum(f * Pxx) / np.sum(Pxx)
    return spectral_centroid


# Calculate Spectral Maximum Peaks
def calculate_spectral_max_peaks(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    peaks, _ = find_peaks(Pxx)
    max_peaks = f[peaks]
    return max_peaks


# Calculate Spectral Roll-off
def calculate_spectral_roll_off(signal, fs, roll_off_percent=0.85):
    f, Pxx = welch(signal, fs=fs)
    cumsum_pxx = np.cumsum(Pxx)
    total_power = cumsum_pxx[-1]
    roll_off_idx = np.argmax(cumsum_pxx >= roll_off_percent * total_power)
    roll_off_freq = f[roll_off_idx]
    return roll_off_freq


# Calculate Median Frequency (MDF)
def calculate_median_frequency(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    cumsum_pxx = np.cumsum(Pxx)
    median_power = cumsum_pxx[-1] / 2
    mdf_idx = np.argmax(cumsum_pxx >= median_power)
    mdf_freq = f[mdf_idx]
    return mdf_freq


# Calculate Max Frequency
def calculate_max_frequency(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    max_freq = f[np.argmax(Pxx)]
    return max_freq


# Calculate Peak Frequency
def calculate_peak_frequency(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    peaks, _ = find_peaks(Pxx)
    peak_freq = f[peaks[np.argmax(Pxx[peaks])]]
    return peak_freq


# Calculate Kurtosis
def calculate_kurtosis(signal):
    kurt = kurtosis(signal)
    return kurt


# Calculate Skewness
def calculate_skewness(signal):
    skewness = skew(signal)
    return skewness


# Calculate Power Bandwidth
def calculate_power_bandwidth(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    total_power = np.sum(Pxx)
    power_bandwidth = np.sum(Pxx >= total_power / 2)
    return power_bandwidth


# Calculate Maximum Power Spectrum
def calculate_max_power_spectrum(signal, fs):
    f, Pxx = welch(signal, fs=fs)
    max_power_spectrum = np.max(Pxx)
    return max_power_spectrum


# # Calculate Human Range Energy
# def calculate_human_range_energy(signal, fs):
#     _, _, Sxx = spectrogram(signal, fs=fs)
#     human_range_frequencies = (20, 20000)
#     human_range_energy = np.sum(Sxx[(frequencies >= human_range_frequencies[0]) & (frequencies <= human_range_frequencies[1])])
#     return human_range_energy


# Calculate Variation
def calculate_variation(signal):
    variation = np.std(signal) / np.mean(signal)
    return variation


# Calculate MFCC
def calculate_mfcc(signal, fs):
    mfcc = psf.mfcc(signal, samplerate=fs)
    return mfcc


# # Calculate LPCC
# def calculate_lpcc(signal, fs):
#     lpcc = psf.(signal, fs=fs)
#     return lpcc